In [1]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from torchinfo import summary
from torchvision import datasets, transforms, models

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


## Implementatation

In [2]:
class SEBlock(nn.Module):
    def __init__(self, channels, reduction_ratio):
        super().__init__()
        rounded_cahnnels = max(8, round((channels // reduction_ratio) + 8 / 2) // 8 * 8)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.linear1 = nn.Linear(channels, rounded_cahnnels)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(rounded_cahnnels, channels)
        self.sigmoid = nn.Hardsigmoid()
    
    def forward(self, x):
        se = self.gap(x)
        se = se.flatten(start_dim=1)
        se = self.linear1(se)
        se = self.relu(se)
        se = self.linear2(se)
        se = self.sigmoid(se)
        se = se[..., None, None]
        x = x * se
        return x

In [3]:
class DepthwiseSeparableConvolution(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, use_se, activation):
        super().__init__()
        self.depthwise_conv = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, stride=stride, padding=(kernel_size - 1) // 2,
                      bias=False,
                      groups=in_channels),
            nn.BatchNorm2d(in_channels),
            nn.ReLU() if activation == 'RE' else nn.Hardswish(),
        )
        self.se_block = SEBlock(in_channels, 4) if use_se else None
        self.seeparable_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
        )
    
    def forward(self, x):
        x = self.depthwise_conv(x)
        x = self.se_block(x) if self.se_block else x
        x = self.seeparable_conv(x)
        return x

In [4]:
class InvertedResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, expand_channels, kernel_size, stride, use_se, activation):
        super().__init__()
        self.use_skip_connection = (stride == 1 and in_channels == out_channels)
        
        layers = [nn.Sequential(
            nn.Conv2d(in_channels, expand_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(expand_channels),
            nn.ReLU() if activation == 'RE' else nn.Hardswish(),
        )] if in_channels != expand_channels else []
        self.bottleneck = nn.Sequential(
            *layers,
            DepthwiseSeparableConvolution(expand_channels, out_channels, kernel_size, stride, use_se, activation)
        )
    
    def forward(self, x):
        bottleneck = self.bottleneck(x)
        bottleneck = bottleneck + x if self.use_skip_connection else bottleneck
        return bottleneck


x_batch = torch.randn(32, 64, 52, 52)
print(InvertedResidualBlock(64, 32, 3, 6, 2, True, 'RE')(x_batch).shape)

torch.Size([32, 32, 26, 26])


In [5]:
class MobileNetV3(nn.Module):
    def __init__(self, n_channels, n_classes):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(n_channels, 16, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.Hardswish()
        )
        self.conv2 = InvertedResidualBlock(16, 16, 16, 3, 2, True, 'RE')
        self.conv3 = InvertedResidualBlock(16, 24, 72, 3, 2, False, 'RE')
        self.conv4 = InvertedResidualBlock(24, 24, 88, 3, 1, False, 'RE')
        self.conv5 = InvertedResidualBlock(24, 40, 96, 5, 2, True, 'HS')
        self.conv6 = InvertedResidualBlock(40, 40, 240, 5, 1, True, 'HS')
        self.conv7 = InvertedResidualBlock(40, 40, 240, 5, 1, True, 'HS')
        self.conv8 = InvertedResidualBlock(40, 48, 120, 5, 1, True, 'HS')
        self.conv9 = InvertedResidualBlock(48, 48, 144, 5, 1, True, 'HS')
        self.conv10 = InvertedResidualBlock(48, 96, 288, 5, 2, True, 'HS')
        self.conv11 = InvertedResidualBlock(96, 96, 576, 5, 1, True, 'HS')
        self.conv12 = InvertedResidualBlock(96, 96, 576, 5, 1, True, 'HS')
        self.conv13 = nn.Sequential(
            nn.Conv2d(96, 576, kernel_size=1, bias=False),
            nn.BatchNorm2d(576),
            nn.Hardswish()
        )
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.conv14 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(576, 1024),
            nn.Hardswish()
        )
        self.classifier = nn.Sequential(
            nn.Linear(1024, n_classes),
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.avgpool(x)
        x = self.conv14(x)
        x = self.classifier(x)
        return x

# print(MobileNetV3(3, 1000))
# x_batch = torch.randn(32, 3, 224, 224)
# print(MobileNetV2(3, 1000)(x_batch).shape)

In [6]:
mobilenet_v3_model = MobileNetV3(3, 1000)
summary(mobilenet_v3_model, input_size=(1, 3, 224, 224), col_names=['output_size', 'num_params', 'mult_adds'],
        device='cpu', depth=5)
# print(mobilenet_v3_model)

Layer (type:depth-idx)                                  Output Shape              Param #                   Mult-Adds
MobileNetV3                                             [1, 1000]                 --                        --
├─Sequential: 1-1                                       [1, 16, 112, 112]         --                        --
│    └─Conv2d: 2-1                                      [1, 16, 112, 112]         432                       5,419,008
│    └─BatchNorm2d: 2-2                                 [1, 16, 112, 112]         32                        32
│    └─Hardswish: 2-3                                   [1, 16, 112, 112]         --                        --
├─InvertedResidualBlock: 1-2                            [1, 16, 56, 56]           --                        --
│    └─Sequential: 2-4                                  [1, 16, 56, 56]           --                        --
│    │    └─DepthwiseSeparableConvolution: 3-1          [1, 16, 56, 56]           --              

In [7]:
mobilenet_v3_torch_model = models.mobilenet_v3_small()
summary(mobilenet_v3_torch_model, input_size=(1, 3, 224, 224), col_names=['output_size', 'num_params', 'mult_adds'],
        device='cpu')
# print(mobilenet_v3_torch_model)

Layer (type:depth-idx)                             Output Shape              Param #                   Mult-Adds
MobileNetV3                                        [1, 1000]                 --                        --
├─Sequential: 1-1                                  [1, 576, 7, 7]            --                        --
│    └─Conv2dNormActivation: 2-1                   [1, 16, 112, 112]         --                        --
│    │    └─Conv2d: 3-1                            [1, 16, 112, 112]         432                       5,419,008
│    │    └─BatchNorm2d: 3-2                       [1, 16, 112, 112]         32                        32
│    │    └─Hardswish: 3-3                         [1, 16, 112, 112]         --                        --
│    └─InvertedResidual: 2-2                       [1, 16, 56, 56]           --                        --
│    │    └─Sequential: 3-4                        [1, 16, 56, 56]           744                       1,254,744
│    └─InvertedResidual: 

## Training

In [8]:
from pathlib import Path

TRAIN_RATIO = 0.8
data_dir = Path('./data/')

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

train_ds = datasets.CIFAR100(data_dir, train=True, download=True, transform=transform)
train_ds, val_ds = random_split(train_ds, (TRAIN_RATIO, 1 - TRAIN_RATIO))
val_ds.transform = transform
test_ds = datasets.CIFAR100(data_dir, train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
import wandb
from src.engine import *

config = dict(batch_size=512, lr=3e-3, epochs=20, dataset='CIFAR100')
with wandb.init(project='pytorch-study', name='MobileNetV3', config=config) as run:
    w_config = run.config
    train_dl = DataLoader(train_ds, batch_size=w_config.batch_size, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=w_config.batch_size, shuffle=True)
    
    n_classes = len(train_ds.dataset.classes)
    mobilenet_v3_model = MobileNetV3(3, n_classes).to(DEVICE)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(mobilenet_v3_model.parameters(), lr=w_config.lr)
    
    loss_history, acc_history = train(mobilenet_v3_model, train_dl, val_dl, criterion, optimizer, w_config.epochs, DEVICE, run) 

Epoch=2:   5%|▌         | 1/20 [02:11<41:33, 131.26s/it, train_loss=3.733, train_acc=11.33%, val_loss=3.414, val_acc=17.13%]
Traceback (most recent call last):
  File "C:\Users\dk\AppData\Local\Temp\ipykernel_27628\3236535813.py", line 16, in <module>
    loss_history, acc_history = train(mobilenet_v3_model, train_dl, val_dl, criterion, optimizer, w_config.epochs, DEVICE, run)
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\OneDrive\local\workspace\paper implementation\src\engine.py", line 50, in train
    train_loss, train_acc = batch_epoch(model, train_dl, criterion, device, optimizer)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\OneDrive\local\workspace\paper implementation\src\engine.py", line 14, in batch_epoch
    y_logits = model(x_batch)
               ^^^^^^^^^^^^^^
  File "C:\Users\dk\.pipenv\paper_implementation-aJVmDThZ\Lib\site-pack

KeyboardInterrupt: 